In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list=[]
    
    positive_path=path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
        
    negative_path=path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
    
    print('read',filetype, 'files:',len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
            
    return all_labels,all_texts

Using TensorFlow backend.


In [2]:
y_train,train_text=read_files("train")

read train files: 25000


In [3]:
y_test,test_text=read_files('test')

read test files: 25000


In [4]:
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)

x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=380)

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

In [7]:
model = Sequential()

In [8]:
model.add(Embedding(output_dim=32,
                   input_dim=3800,
                   input_length=380))
model.add(Dropout(0.2))

In [9]:
model.add(LSTM(32))

In [10]:
model.add(Dense(units=256,
               activation='relu'))
model.add(Dropout(0.2))

In [11]:
model.add(Dense(units=1,
               activation='sigmoid'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 138,625
Trainable params: 138,625
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [16]:
train_history = model.fit(x_train, y_train, batch_size=100,
                         epochs=10,verbose=2,
                         validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 64s - loss: 0.5136 - acc: 0.7334 - val_loss: 0.3783 - val_acc: 0.8132
Epoch 2/10
 - 63s - loss: 0.2821 - acc: 0.8861 - val_loss: 0.4004 - val_acc: 0.8208
Epoch 3/10
 - 59s - loss: 0.2234 - acc: 0.9124 - val_loss: 0.4658 - val_acc: 0.8118
Epoch 4/10
 - 63s - loss: 0.1995 - acc: 0.9234 - val_loss: 0.3921 - val_acc: 0.8524
Epoch 5/10
 - 70s - loss: 0.1748 - acc: 0.9344 - val_loss: 0.4724 - val_acc: 0.8106
Epoch 6/10
 - 64s - loss: 0.1547 - acc: 0.9433 - val_loss: 0.3343 - val_acc: 0.8650
Epoch 7/10
 - 63s - loss: 0.1488 - acc: 0.9428 - val_loss: 0.5350 - val_acc: 0.8314
Epoch 8/10
 - 68s - loss: 0.1317 - acc: 0.9521 - val_loss: 0.6880 - val_acc: 0.7872
Epoch 9/10
 - 58s - loss: 0.1171 - acc: 0.9574 - val_loss: 0.6491 - val_acc: 0.8186
Epoch 10/10
 - 57s - loss: 0.1142 - acc: 0.9593 - val_loss: 0.7666 - val_acc: 0.8042


In [17]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 28s 1ms/step


0.85240000000000005